In [110]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [111]:
def accept_cookies(driver):
    """Akzeptiert das Cookie-Banner wenn vorhanden."""
    try:
        wait = WebDriverWait(driver, 10)
        cookie_button = wait.until(
            EC.element_to_be_clickable(
                (By.ID, "onetrust-accept-btn-handler")
            )
        )
        cookie_button.click()
        print("Cookie-Banner akzeptiert")
        time.sleep(1)  # Kurze Pause nach Klick
    except TimeoutException:
        print("Kein Cookie-Banner gefunden oder bereits akzeptiert")

def fix_umlauts(text):
    """Korrigiert falsch kodierte Umlaute."""
    replacements = {
        'Ã¤': 'ä', 'Ã„': 'Ä', 'Ã¶': 'ö', 'Ã–': 'Ö',
        'Ã¼': 'ü', 'Ãœ': 'Ü', 'ÃŸ': 'ß'
    }
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    return text

def slugify(text: str) -> str:
    """Wandelt Text in URL-Slug um."""
    text = fix_umlauts(text.lower())
    replacements = {
        'ä': 'ae', 'ö': 'oe', 'ü': 'ue', 'ß': 'ss',
        ' ': '-', '/': '-'
    }
    for orig, repl in replacements.items():
        text = text.replace(orig, repl)
    return text

def parse_salary(pill_text):
    """Extrahiert Gehaltsangaben."""
    clean_text = pill_text.replace('\xa0', ' ').strip()
    pattern = r"""(?:ab\s*)?(\d{1,3}(?:\.?\d{3})*(?:,\d{1,2})?)\s*€\s*(?:[-–—]\s*(\d{1,3}(?:\.?\d{3})*(?:,\d{1,2})?)\s*€\s*)?(monatlich|jährlich)?"""
    match = re.search(pattern, clean_text, re.IGNORECASE | re.VERBOSE)
    if not match:
        return "keine Angabe"

    try:
        def parse_euro_number(num_str):
            return float(num_str.replace('.', '').replace(',', '.'))

        lower = parse_euro_number(match.group(1))
        upper = parse_euro_number(match.group(2)) if match.group(2) else lower
        avg_salary = (lower + upper) / 2
        zeitraum = match.group(3).lower() if match.group(3) else "jährlich"

        if zeitraum == "monatlich":
            avg_salary *= 14

        return str(round(avg_salary)) if avg_salary >= 15000 else "keine Angabe"
    except:
        return "keine Angabe"

def extract_city(job_element, target_city):
    """Extrahiert die Stadt aus dem Job-Element."""
    location_container = job_element.find('span', class_='m-jobsListItem__locations')
    if location_container:
        links = location_container.find_all('a', class_='m-jobsListItem__location')
        for link in links:
            text = link.get_text(strip=True).lower()
            if target_city in text:
                return target_city.capitalize()
    return None

def click_load_more(driver):
    """Klickt den 'Weitere Jobs anzeigen'-Button."""
    try:
        wait = WebDriverWait(driver, 10)
        button = wait.until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "button.m-loadMoreJobsButton__button")
            )
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", button)
        time.sleep(3)
        return True
    except (TimeoutException):
        return False


In [112]:
def scrape_jobs_karriere(driver, query="data-analyst", locations=["wien"], max_attempts=10):
    """Hauptfunktion zum Scrapen von Jobs."""
    jobs = []
    query_slug = slugify(query)

    for city_slug in locations:
        print(f"\nStarte Suche für {city_slug.capitalize()}...")
        url = f"https://www.karriere.at/jobs/{query_slug}/{city_slug}"
        driver.get(url)

        # Warte auf Seitenladevorgang und akzeptiere Cookies
        time.sleep(5)
        accept_cookies(driver)

        # Lade alle verfügbaren Jobs
        attempt = 0
        while attempt < max_attempts:
            if not click_load_more(driver):
                break
            attempt += 1
            print(f"Lade weitere Jobs... Versuch {attempt}/{max_attempts}")

        # Parse die Jobdaten
        soup = BeautifulSoup(driver.page_source, "html.parser")
        job_items = soup.find_all("div", class_="m-jobsListItem")
        print(f"Gefundene Jobs für {city_slug.capitalize()}: {len(job_items)}")

        for job in job_items:
            title_tag = job.find("a", class_="m-jobsListItem__titleLink")
            job_title = fix_umlauts(title_tag.get_text(strip=True)) if title_tag else "Kein Jobtitel"

            city = extract_city(job, city_slug)
            if not city:
                continue

            salary_annual = "keine Angabe"
            salary_pills = job.find_all("span", class_=lambda x: x and "m-jobsListItem__pill" in x)
            for pill in salary_pills:
                pill_text = pill.get_text(strip=True)
                if "€" in pill_text:
                    salary_annual = parse_salary(pill_text)
                    break

            jobs.append({
                "job_title": job_title,
                "city": city,
                "country": "Austria",
                "annual_salary": salary_annual
            })

    return jobs


In [113]:
# Hauptprogramm
if __name__ == "__main__":
    options = Options()
    options.headless = False  # Für Debugging sichtbar machen
    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=options)

    try:
        cities = ["wien", "linz", "graz", "salzburg", "innsbruck"]

        jobs = scrape_jobs_karriere(
            driver,
            query="Data Engineer",
            locations=cities,
            max_attempts=10
        )

        df = pd.DataFrame(jobs)
        print(f"\nGesamtanzahl gescrapte Jobs: {len(df)}")
        print(df.head())

        df.to_csv("../output/data_analyst_engineer_scientist_austria.csv", index=False, encoding="utf-8-sig")
        print("Ergebnisse gespeichert in 'data_analyst_engineer_scientist_austria.csv'")

    finally:
        driver.quit()


Starte Suche für Wien...
Kein Cookie-Banner gefunden oder bereits akzeptiert
Lade weitere Jobs... Versuch 1/10
Lade weitere Jobs... Versuch 2/10
Lade weitere Jobs... Versuch 3/10
Gefundene Jobs für Wien: 59

Starte Suche für Linz...
Kein Cookie-Banner gefunden oder bereits akzeptiert
Gefundene Jobs für Linz: 15

Starte Suche für Graz...
Kein Cookie-Banner gefunden oder bereits akzeptiert
Lade weitere Jobs... Versuch 1/10
Gefundene Jobs für Graz: 26

Starte Suche für Salzburg...
Kein Cookie-Banner gefunden oder bereits akzeptiert
Gefundene Jobs für Salzburg: 15

Starte Suche für Innsbruck...
Kein Cookie-Banner gefunden oder bereits akzeptiert
Gefundene Jobs für Innsbruck: 15

Gesamtanzahl gescrapte Jobs: 100
                                         job_title  city  country  \
0                          Splunk Engineer (m/w/d)  Wien  Austria   
1  Systems Engineer IT-Security Operations (m/w/d)  Wien  Austria   
2      Systems Engineer Network Operations (m/w/d)  Wien  Austria   
3     